In [1034]:
import numpy as np
import math
from scipy.stats import norm
from sklearn.metrics import accuracy_score


In [1035]:
obs = np.loadtxt("data.txt", dtype=float) 

In [1036]:
obs = obs.reshape(-1,1)

# obs[2][0]

In [1037]:
paramfile = "parameters.txt"
states = 0
with open(paramfile) as f:
    states = int(f.readline().strip())


trans_p = np.loadtxt(paramfile , dtype = float , skiprows = 1 , max_rows = states )

means = np.loadtxt(paramfile , dtype = float , skiprows = states + 1  , max_rows = 1 ).reshape(-1,1)

sd = np.loadtxt(paramfile , dtype = float , skiprows = states + 2  , max_rows = 1 ).reshape(-1,1)
# trans_p
# means[1][0]

In [1038]:
b = np.zeros([states,1] , float)
b[states-1][0] = 1

trans_p_copy = np.copy(trans_p)
a = trans_p_copy[:,:states-1]


for i in range(0,states-1):
    a[i][i] = a[i][i] - 1

a = np.vstack((a , np.ones([states,1]))).reshape(states,states)

start_p = np.linalg.solve(a , b)


In [1039]:
def normal_dist(x , mean , sd):
    prob_density = norm.pdf(x, loc=mean, scale=np.sqrt(sd))
    if prob_density == 0:
        prob_density = 1e-250
    return prob_density

In [1040]:
def viterbi():

    problist = np.zeros([len(obs), states])
    prevlist = np.zeros([len(obs), states])

    for state in range(states):
        problist[0][state] = np.log(start_p[state] * normal_dist(obs[0][0] , means[0][0], sd[0][0]))
        prevlist[0][state] = -1

  
    for t in range(1, len(obs)):
        
        for state in range(states):
            max_tr_prob = problist[t-1][0] + np.log(trans_p[0][state])
            prev_state_selected = 0

            for prev_state in range(1,states):
                tr_prob = problist[t - 1][prev_state] + np.log(trans_p[prev_state][state])
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_state_selected = prev_state

            
            max_prob = max_tr_prob + np.log(normal_dist(obs[t][0] , means[state][0] , sd[state][0]))
            problist[t][state] = max_prob
            prevlist[t][state] = prev_state_selected


    opt = []
    max_prob = float('-inf')
    best_state = -1
    # Get most probable state and its backtrack
    for state in range(states):
        if problist[len(obs)-1][state] > max_prob:
            max_prob = problist[len(obs)-1][state]
            best_state = state


    opt.append(best_state)
    previous = best_state

    
    for t in range(len(problist) - 2, -1, -1):
        opt.insert(0, prevlist[t + 1][previous])
        previous = int(prevlist[t + 1][previous])


    opt = [int(x) for x in opt]

    return opt
    # print(opt)    



In [1041]:

output = viterbi()
# print(output.shape)

output_list = ['"El Nino"' if output[i] == 0 else '"La Nina"' for i in range(len(output))]

with open('states_Viterbi_wo_learning.txt', 'w') as f:
    for item in output_list:
        f.write("%s\n" % item)



with open('sir.txt') as f:
    sirer_output_list = [line.rstrip() for line in f]

accuracy_score(sirer_output_list , output_list)

# sirer_output_list

# output_list

1.0